In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

# keras
from keras import Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import Callback

# custom define
from src.load_data import get_read_main
from src.train import create_model_conv, training

# jupyter themes
from jupyterthemes import jtplot
jtplot.style('monokai')

os.environ["CUDA_VISIBLE_DEVICES"]="0,2"

# 讀取資料

In [ ]:
X_train, X_test, y_train, y_test = get_read_main('./characters/', save=False, load=True)
print("Train sample:", X_train.shape[0])
print("Test sample:", X_test.shape[0])

# 建構模型

## Sequential模型

In [ ]:
model = create_model_conv(input_shape=X_train.shape[1:], num_classes=20, show_summary=False)

## Multi GPU模型

In [ ]:
class ModelMGPU(Model):
    def __init__(self, ser_model, gpus):
        pmodel = multi_gpu_model(ser_model, gpus)
        self.__dict__.update(pmodel.__dict__)
        self._smodel = ser_model

    def __getattribute__(self, attrname):
        '''Override load and save methods to be used from the serial-model. The
        serial-model holds references to the weights in the multi-gpu model.
        '''
        # return Model.__getattribute__(self, attrname)
        if 'load' in attrname or 'save' in attrname:
            return getattr(self._smodel, attrname)

        return super(ModelMGPU, self).__getattribute__(attrname)

In [ ]:
parallel_model = ModelMGPU(model , 2)

## compile
在訓練模型之前，您需要配置學習過程，這是通過編譯方法完成的。它接收三個參數：
+ 優化器 optimizer
+ 損失函數 loss
+ 評估指標 metrics

In [ ]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
parallel_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# 訓練模型

## Callbacks

In [ ]:
class LossHistory(Callback):
    def on_train_begin(self, logs=None):
        self.loss = []
        self.acc = []
        self.val_loss = []
        self.val_acc = []
    def on_batch_end(self, batch, logs=None):
        self.loss.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))
        self.val_loss.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_acc'))

loss_history = LossHistory()
checkpoint = ModelCheckpoint('best_weights.hdf5',
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [ ]:
cb_list = [checkpoint]

# Start training
model, history = training(parallel_model, X_train, X_test, y_train, y_test,
                          batch_size=1024, epochs=100, callbacks_list=cb_list)

In [ ]:
# summarize history for accuracy
fig = plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig('curve.png')